In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df_categories = pd.read_csv("products_categories.csv", sep=",", encoding='latin-1')

In [3]:
df_categories.head()

,sku,cat1,cat2,cat3
0,00.01.10.014,Cosmética y Belleza,Corporal,Hidratación
1,00.071697.000.000,Infantil,Juguetes,Mordedores
2,000133,Infantil,Higiene infantil,Canastillas y kits bebé
3,000147,Higiene y cuidado personal,Facial,Desmaquillantes y limpiadores
4,000148,Cosmética y Belleza,Manos,Crema de Manos


In [4]:
# df_categories[df_categories["cat1"].isna()]

In [5]:
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11159 entries, 0 to 11158
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sku     11159 non-null  object
 1   cat1    10741 non-null  object
 2   cat2    10721 non-null  object
 3   cat3    10441 non-null  object
dtypes: object(4)
memory usage: 348.8+ KB


In [6]:
categories_na = df_categories[df_categories["cat1"].isna()]

In [8]:
categories_na.head()

,sku,cat1,cat2,cat3
30,"001268,Salud,""Huesos, articulaciones y músculo...",NaN,NaN,NaN
44,"002655,Salud,Botiquín,""Apósitos, tiritas y gasas""",NaN,NaN,NaN
49,"002870,Salud,""Huesos, articulaciones y músculo...",NaN,NaN,NaN
50,"002872,Salud,""Huesos, articulaciones y músculo...",NaN,NaN,NaN
51,"002873,Salud,""Huesos, articulaciones y músculo...",NaN,NaN,NaN


In [9]:
categories_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 30 to 11082
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sku     418 non-null    object
 1   cat1    0 non-null      object
 2   cat2    0 non-null      object
 3   cat3    0 non-null      object
dtypes: object(4)
memory usage: 16.3+ KB


In [10]:
categories_sep = categories_na["sku"].str.split(",", expand=True)

In [11]:
categories_sep = categories_sep.rename(columns={0: "sku", 1: "cat1"})

In [12]:
categories_sep["cat2,3"] = categories_sep[[2,3,4]].apply(", ".join, axis=1)

In [13]:
categories_sep = categories_sep.drop(labels=[2,3,4], axis=1)

In [14]:
categories_sep.head()

,sku,cat1,"cat2,3"
30,001268,Salud,"""Huesos, articulaciones y músculos"", Colágeno"
44,002655,Salud,"Botiquín, ""Apósitos, tiritas y gasas"""
49,002870,Salud,"""Huesos, articulaciones y músculos"", Colágeno"
50,002872,Salud,"""Huesos, articulaciones y músculos"", Colágeno"
51,002873,Salud,"""Huesos, articulaciones y músculos"", Colágeno"


In [15]:
categories_sep_2_3 = categories_sep["cat2,3"].str.split('"', expand=True)

In [16]:
categories_sep_2_3 = categories_sep_2_3.rename(columns={0: "cat2", 1: "cat3"})

In [18]:
categories_sep_2_3.head()

,cat2,cat3,2
30,,"Huesos, articulaciones y músculos",", Colágeno"
44,"Botiquín,","Apósitos, tiritas y gasas",
49,,"Huesos, articulaciones y músculos",", Colágeno"
50,,"Huesos, articulaciones y músculos",", Colágeno"
51,,"Huesos, articulaciones y músculos",", Colágeno"


In [20]:
categories_sep_2_3.head()

,cat2,cat3,2
30,,"Huesos, articulaciones y músculos",", Colágeno"
44,"Botiquín,","Apósitos, tiritas y gasas",
49,,"Huesos, articulaciones y músculos",", Colágeno"
50,,"Huesos, articulaciones y músculos",", Colágeno"
51,,"Huesos, articulaciones y músculos",", Colágeno"


In [21]:
for pos, na in enumerate(categories_sep_2_3["cat2"]):
    if na == "":
        categories_sep_2_3["cat2"].iloc[pos] = categories_sep_2_3["cat3"].iloc[pos]
        categories_sep_2_3["cat3"].iloc[pos] = categories_sep_2_3[2].iloc[pos]

categories_sep_2_3.drop(columns=[2], inplace=True)

In [22]:
categories_sep_2_3["cat3"]=categories_sep_2_3["cat3"].str.replace("^[,]\s", "", regex=True)
categories_sep_2_3["cat2"]=categories_sep_2_3["cat2"].str.replace("[,]\s$", "", regex=True)

In [24]:
categories_sep_2_3.head()

,cat2,cat3
30,"Huesos, articulaciones y músculos",Colágeno
44,Botiquín,"Apósitos, tiritas y gasas"
49,"Huesos, articulaciones y músculos",Colágeno
50,"Huesos, articulaciones y músculos",Colágeno
51,"Huesos, articulaciones y músculos",Colágeno


In [25]:
categories_sep_clean = pd.merge(categories_sep, categories_sep_2_3, left_index=True, right_index=True)
categories_sep_clean.drop(columns="cat2,3", inplace=True)

In [27]:
categories_sep_clean.head()

,sku,cat1,cat2,cat3
30,001268,Salud,"Huesos, articulaciones y músculos",Colágeno
44,002655,Salud,Botiquín,"Apósitos, tiritas y gasas"
49,002870,Salud,"Huesos, articulaciones y músculos",Colágeno
50,002872,Salud,"Huesos, articulaciones y músculos",Colágeno
51,002873,Salud,"Huesos, articulaciones y músculos",Colágeno


In [28]:
df_categories.drop(df_categories[df_categories["cat1"].isna()].index, inplace=True)

In [29]:
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10741 entries, 0 to 11158
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sku     10741 non-null  object
 1   cat1    10741 non-null  object
 2   cat2    10721 non-null  object
 3   cat3    10441 non-null  object
dtypes: object(4)
memory usage: 419.6+ KB


In [30]:
# for i in df_categories["cat1"]:
#     if i is na:
#         df_categories["cat1"][i].drop(axis=0, inplace=True)

In [31]:
df_categories["cat1"].isna().sum()

0

In [32]:
df_categories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10741 entries, 0 to 11158
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sku     10741 non-null  object
 1   cat1    10741 non-null  object
 2   cat2    10721 non-null  object
 3   cat3    10441 non-null  object
dtypes: object(4)
memory usage: 419.6+ KB


In [33]:
df_categories_limpieza1 = pd.concat([df_categories, categories_sep_clean])

In [34]:
# df_categories_limpieza1 = df_categories.append(categories_sep_clean, ignore_index=True)

In [35]:
df_categories_limpieza1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11159 entries, 0 to 11082
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sku     11159 non-null  object
 1   cat1    11159 non-null  object
 2   cat2    11139 non-null  object
 3   cat3    10859 non-null  object
dtypes: object(4)
memory usage: 435.9+ KB


In [37]:
df_categories_limpieza1[df_categories_limpieza1["cat2"].isna()].head()

,sku,cat1,cat2,cat3
31,001469,Veterinaria,NaN,NaN
743,113159=575349,Veterinaria,NaN,NaN
8172,570052,Veterinaria,NaN,NaN
8173,570235,Veterinaria,NaN,NaN
8174,570237,Veterinaria,NaN,NaN


In [38]:
df_categories["cat1"] = df_categories["cat1"].str.lower()
df_categories["cat2"] = df_categories["cat2"].str.lower()
df_categories["cat3"] = df_categories["cat3"].str.lower()


In [39]:
df_categories_limpieza1.to_csv("categories_limpieza1.csv")

In [40]:
# categories_na.to_csv("categories_na.csv")

In [41]:
# categories_na_df = pd.read_csv("categories_na.csv", sep=',')

In [42]:
# categories_na_df